In [14]:
import sys 
sys.path.append("../0-Functions")
import numpy as np

from Graphs_functions import trends_month
from Graphs_functions import cdf_plot_period
from Functions_ImportData import import_filtered_NOAA_obs
from Functions_ImportData import import_gorongosa_obs_pr
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6

In [2]:
from Functions_Indicators import filter_dataframe

In [3]:
# to do
# pr: season historique, dry, wet, past, all scenarios
# tas: season historique, dry, wet, past, all scenarios
# tasmax: season historique, dry, wet, past, all scenarios

In [4]:
# observation data
obs_NOAA=import_filtered_NOAA_obs()
pr_obs= import_gorongosa_obs_pr()

In [5]:
# modelled
tas=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA('tas', 'Celsius')
tasmax=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA('tasmax', 'Celsius')
tasmin=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA('tasmin', 'Celsius')
pr=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA('pr', 'mm_per_day')
pr_proj=import_treat_modeled_NEX_GDDP_CMIP6('pr', 'mm_per_day','day',1950,2100)

In [6]:
# filter data

In [7]:
tas_chimoio=filter_dataframe(tas,['CHIMOIO, MZ'],['TaiESM1','NESM3','CMCC-CM2-SR5'])
tasmax_chimoio=filter_dataframe(tasmax,['CHIMOIO, MZ'],['TaiESM1','NESM3','CMCC-CM2-SR5'])
tasmin_chimoio=filter_dataframe(tasmin,['CHIMOIO, MZ'],['TaiESM1','NESM3','CMCC-CM2-SR5'])
pr_chimoio=filter_dataframe(pr,['CHIMOIO, MZ'],['TaiESM1','NESM3','CMCC-CM2-SR5'])
pr_proj_gorongosa=filter_dataframe(pr_proj,['Gorongosa_EIB'],['TaiESM1','NESM3','CMCC-CM2-SR5'])

In [8]:
obs_NOAA_chimoio=obs_NOAA[obs_NOAA['NAME']=='CHIMOIO, MZ']

In [9]:
# graphs

In [ ]:
# trends_month

In [ ]:
pr_obs.rename(columns = {'pr':'PRCP'}, inplace = True)

In [ ]:
trends_month('pr',pr_proj_gorongosa,'NEX-GDDP-CMIP6',pr_obs,'NOAA','Average','Gorongosa',temporal_resolution='Month',start_year_line=1970,stop_year_line=2014,start_year_boxplot=1950,stop_year_boxplot=2100,dry_season='no')

In [ ]:
trends_month('pr',pr_proj_gorongosa,'NEX-GDDP-CMIP6',pr_obs,'NOAA','Average','Gorongosa',temporal_resolution='Month',start_year_line=1970,stop_year_line=2014,start_year_boxplot=1950,stop_year_boxplot=2100,dry_season='yes')

In [ ]:
# cdf

In [11]:
def cdf_plot_period(df,periods,name_column,df_obs,name_col_obs):

    df_copy = df.copy(deep=True) # do a copy of the dataframe
    # create two new columns to register the periods and cdfs
    df_copy['Period'] = df_copy['Year']
    df_copy['CDF'] = df_copy['Year']
    # register historical information to concatenate them with each period
    df_historical = df_copy[df_copy['Experiment']=='historical']
    df_historical=cdf_(df_historical,name_column)
    df_obs.loc[:,'Experiment']='Observation'
    df_obs=cdf_(df_obs,name_col_obs)
    # create an empty dataframe for the information to plot
    df_final=pd.DataFrame()
    for period in periods: # go throught all the periods wnated
        df_temp = df_copy[df_copy['Year'].between(int(period[0:4]),int(period[5:9]))]
        df_temp.loc[:,'Period'] = period
        df_historical.loc[:,'Period'] = period
        df_obs.loc[:,'Period'] = period
        df_ssp = pd.DataFrame()
        for ssp in list(set(df['Experiment'])): # go throught all the spp wanted
            # select the ssp and sort the data
            df_temp_ssp = df_temp[df_temp['Experiment']==ssp]#.sort_values(name_column,na_position='first').dropna()# sort value in cdf_ function
            # calulate the cdf
            df_temp_ssp=cdf_(df_temp_ssp,name_column)
            # concat the result with the other ssps
            df_ssp = pd.concat([df_ssp,df_temp_ssp])
        # concat the results with the other ssps and periods
        df_final = pd.concat([df_final,df_obs,df_historical,df_ssp])
        
    # plot the different periods with the historical data
    
    Palette = ["black", "blue","red", "green","orange"]
    sns.set_palette(Palette) #use the list defined in the function
    g = sns.FacetGrid(df_final, col="Period", height=3, aspect=.8)
    g.map(sns.lineplot, name_column,'CDF','Experiment')
    g.add_legend(title='Legend')
    g.fig.subplots_adjust(top=0.75) # adjust the Figure in rp
    g.fig.suptitle('Cumulative distribution function of the yearly average temperature\nof the modeled data NEX-GDDP-CMIP6')
    g.fig.tight_layout() # Adjust the padding between and around subplots.
    sns.move_legend(g, "right", bbox_to_anchor=(1.25, .45))#, frameon=False)
    plt.show()

    return df_final


# In[ ]:


# the dataframe given in this function should have a column named 'CDF'
def cdf_(df,name_column_df):
    df=df.sort_values(name_column_df,na_position='first').dropna() # sort the values
    df['CDF'] = np.arange(len(df[name_column_df])) / float(len(df[name_column_df]))
    return df

In [ ]:
cdf_plot_period(tasmax_chimoio,['2020-2040','2040-2060'],'Daily Maximum Near-Surface Air Temperature °C',obs_NOAA_chimoio,'TMAX')